# Style-Preserving Speech-to-Speech Translation Experiment

This notebook runs the experiment to determine the minimal duration of speaker embeddings required to effectively clone a speaker's voice across languages.

## 1. Setup Environment
Install necessary dependencies if running on Google Colab.

In [1]:
#only if needed, clear all files except experiment.ipynb
# This command will remove all files and folders in the current directory except "experiment.ipynb"
import os

for fname in os.listdir():
    if fname != "experiment.ipynb":
        if os.path.isdir(fname):
            import shutil
            shutil.rmtree(fname)
        else:
            os.remove(fname)







In [2]:
# Cell to refresh code from GitHub
import os

# Navigate to the repo directory
if os.path.exists("CS479-SpeakerEmbeddings"):
    os.chdir("CS479-SpeakerEmbeddings")
    !git pull
else:
    !git clone https://github.com/NathanAsayDong/CS479-SpeakerEmbeddings.git
    os.chdir("CS479-SpeakerEmbeddings")

# Optional: Reload modules if you've already imported them
import sys
import importlib

# List of your custom modules to reload
modules_to_reload = [
    "common_voice_dataset",
    "setup_experiment",
    "run_experiment",
    "asr_service",
    "translation_service",
    "tts_service",
    "embedding_service",
    "synthetic_data_service",
    "enums"
]

for module_name in modules_to_reload:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
        print(f"Reloaded {module_name}")

Cloning into 'CS479-SpeakerEmbeddings'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 92 (delta 47), reused 71 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 608.14 KiB | 3.10 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [ ]:

# %cd CS479-SpeakerEmbeddings
# !ls

[Errno 2] No such file or directory: 'CS479-SpeakerEmbeddings'
/content
asr_service.py		 peoples_speech_dataset.py  setup_experiment.py
common_voice_dataset.py  ProjectOutline.pdf	    synthetic_data_service.py
embedding_service.py	 __pycache__		    tmp_model
enums.py		 readMe			    translation_service.py
experiment.ipynb	 requirements.txt	    tts_service.py
libri_speech_dataset.py  run_experiment.py
main.py			 Samples


In [4]:
!pip install torch transformers speechbrain soundfile librosa openai-whisper accelerate sentencepiece pydantic torchcodec datasets kagglehub[pandas-datasets]
!pip install sounddevice
!sudo apt-get install libportaudio2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 18.4 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 63.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=443924817c037a9369c0e2fb4fa1502aba86fd7213dd7811267347045266a3c4
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  lib

## 2. Import Modules
Import the experiment setup and runner classes.

In [5]:
import os
import sys

# Add current directory to path if needed
sys.path.append(os.getcwd())

from enums import Language
from setup_experiment import ExperimentSetup
from run_experiment import ExperimentRunner

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)
DEBUG:speechbrain.utils.checkpoints:Regi

## 3. Configure Experiment
Define the parameters for the experiment: source/target languages and reference durations to test.

In [6]:
SOURCE_LANG = Language.ENGLISH
TARGET_LANG = Language.SPANISH
DURATIONS = [5.0, 10.0, 15.0, 20.0, 30.0]
NUM_SAMPLES_PER_DURATION = 5 # Number of unique speakers to test
SEED = 42

## 4. Prepare Data
This step:
1. Downloads/Loads Common Voice dataset via KaggleHub.
2. Selects `NUM_SPEAKERS` with sufficient data.
3. Creates concatenated reference audio files for each duration.
4. Generates a manifest for the experiment run.

In [7]:
!ls

asr_service.py		 peoples_speech_dataset.py  setup_experiment.py
common_voice_dataset.py  ProjectOutline.pdf	    synthetic_data_service.py
embedding_service.py	 __pycache__		    tmp_model
enums.py		 readMe			    translation_service.py
experiment.ipynb	 requirements.txt	    tts_service.py
libri_speech_dataset.py  run_experiment.py
main.py			 Samples


In [8]:
setup = ExperimentSetup(
    source_language=SOURCE_LANG,
    target_language=TARGET_LANG,
    reference_durations=DURATIONS,
    seed=SEED
)

# Prepare the manifest
manifest = setup.prepare_data(num_samples_per_duration=NUM_SAMPLES_PER_DURATION)

print(f"Manifest ready with {len(manifest)} speakers.")
print("Sample Item:", manifest[0] if manifest else "No data")

Preparing experiment data: 5 samples for each of [5.0, 10.0, 15.0, 20.0, 30.0]s durations...
Loading Common Voice dataset for language 'en'...
Using Colab cache for faster access to the 'common-voice' dataset.
Dataset path: /kaggle/input/common-voice
Searching for language 'en' in /kaggle/input/common-voice
Found flattened dataset structure at /kaggle/input/common-voice
Loaded 4076 records for en/dev
Columns: ['filename', 'text', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'duration', 'path', 'sentence']
Manifest ready with 25 speakers.
Sample Item: {'sample_id': 'sample_1', 'test_input_path': '/kaggle/input/common-voice/cv-valid-dev/cv-valid-dev/sample-001749.mp3', 'test_input_text': 'and in that way the months passed', 'reference_path': 'experiment_data/sample_1/ref_5s.wav', 'target_duration': 5.0}


## 5. Run Experiment
Execute the pipeline for each speaker and duration:
1. Extract ground truth embedding (original speaker).
2. Translate source text to Spanish.
3. Synthesize Spanish speech using the reference audio (5s, 10s, etc.) for style.
4. Compute Cosine Similarity between ground truth and output embeddings.

In [ ]:
runner = ExperimentRunner()
runner.run(manifest)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

## 6. Analyze Results
Save and inspect the results.

In [33]:
runner.save_results("experiment_results.csv")

import pandas as pd
results_df = pd.read_csv("experiment_results.csv")

# Display average similarity score per duration
print("\nAverage Similarity Scores by Duration:")
print(results_df.groupby("duration")["similarity_score"].mean())

results_df.head(10)

Results saved to experiment_results.csv

Average Similarity Scores by Duration:
duration
5.0     0.942824
10.0    0.948505
15.0    0.945018
20.0    0.945538
30.0    0.946650
Name: similarity_score, dtype: float64


,sample_id,duration,similarity_score,original_text,target_text,transcribed_text,output_path
0,sample_1,5.0,0.962749,and in that way the months passed,y así pasaron los meses,B.S. Pesarron Los Mises,experiment_data/sample_1/out_sample_1_5.0s.wav
1,sample_1,10.0,0.958668,and in that way the months passed,y así pasaron los meses,Kuay omega ahora. Perse的唱程 Weird countries hac...,experiment_data/sample_1/out_sample_1_10.0s.wav
2,sample_1,15.0,0.962082,and in that way the months passed,y así pasaron los meses,"rhetoric por esposo, wa ng tactics",experiment_data/sample_1/out_sample_1_15.0s.wav
3,sample_1,20.0,0.965659,and in that way the months passed,y así pasaron los meses,ioneserson Überittens,experiment_data/sample_1/out_sample_1_20.0s.wav
4,sample_1,30.0,0.964190,and in that way the months passed,y así pasaron los meses,Basas,experiment_data/sample_1/out_sample_1_30.0s.wav
5,sample_2,5.0,0.930388,about time one of you lunkheads said it,"Ya era hora de que lo dijera uno de ustedes, t...","Ya era horror de Culo Dijera, a no de Estidios...",experiment_data/sample_2/out_sample_2_5.0s.wav
6,sample_2,10.0,0.943128,about time one of you lunkheads said it,"Ya era hora de que lo dijera uno de ustedes, t...","Ya era hora de culo de Dijera, a no de Estid's...",experiment_data/sample_2/out_sample_2_10.0s.wav
7,sample_2,15.0,0.931522,about time one of you lunkheads said it,"Ya era hora de que lo dijera uno de ustedes, t...","¡Ya he ahorado el juventud enjoyable, finalmen...",experiment_data/sample_2/out_sample_2_15.0s.wav
8,sample_2,20.0,0.933768,about time one of you lunkheads said it,"Ya era hora de que lo dijera uno de ustedes, t...","Ya era hora de culo de Jirra, a no de Estid's ...",experiment_data/sample_2/out_sample_2_20.0s.wav
9,sample_2,30.0,0.932847,about time one of you lunkheads said it,"Ya era hora de que lo dijera uno de ustedes, t...","Ya era hora de culo de Dijera, a no de Estid's...",experiment_data/sample_2/out_sample_2_30.0s.wav


## 7. Real-World Demo
Record your own voice, translate it, and check the similarity score.
NOTE: This requires a microphone. If running on a remote Colab kernel without audio forwarding, this might not work directly.

In [34]:
# Real-World Test
from asr_service import ASRService
from translation_service import TranslationService
from tts_service import TTSService
from embedding_service import EmbeddingService
from enums import Language
import torch.nn.functional as F

def run_live_demo(duration=10):
    print(f"--- Live Demo (Enrollment: {duration}s) ---")
    
    # Initialize services
    asr = ASRService()
    translator = TranslationService()
    tts = TTSService()
    embedder = EmbeddingService()
    
    # 1. Record Input
    try:
        # Step 1: Listen and Transcribe
        print("\nRecording... (speak now)")
        english_text, source_audio_path = asr.listen_transcribe(duration=5)
        print(f"Transcribed: {english_text}")
        
        # Step 2: Record Enrollment (Reference)
        # Ideally we use the same audio if it's long enough, but let's record a dedicated style clip
        # input("Press Enter to record style enrollment (speak clearly)...")
        # ref_path = asr.record_audio(duration=duration, file_path="demo_ref.wav")
        
        # Simpler: Just use the input audio itself as reference (Zero-Shot on self)
        ref_path = source_audio_path

        # 2. Translate
        spanish_text = translator.translate(english_text, target_language=Language.SPANISH)
        print(f"Translated: {spanish_text}")

        # 3. Synthesize
        output_path = "demo_output.wav"
        tts.synthesize(spanish_text, output_path, ref_path)
        print(f"Synthesized audio saved to {output_path}")

        # 4. Evaluate Similarity
        gt_embedding = embedder.extract_embedding(ref_path)
        out_embedding = embedder.extract_embedding(output_path)
        
        if gt_embedding.dim() == 1: gt_embedding = gt_embedding.unsqueeze(0)
        if out_embedding.dim() == 1: out_embedding = out_embedding.unsqueeze(0)
            
        score = F.cosine_similarity(gt_embedding, out_embedding).item()
        print(f"Speaker Similarity Score: {score:.4f}")
        
        # Playback (if in Colab/Jupyter)
        from IPython.display import Audio, display
        print("Original:")
        display(Audio(source_audio_path))
        print("Synthesized (Spanish):")
        display(Audio(output_path))
        
    except Exception as e:
        print(f"Error during demo: {e}")
        print("Note: Microphone recording might fail on remote servers without audio forwarding.")

# run_live_demo(duration=10)
# To run this on Colab, you'd typically need a Javascript helper to record audio from the browser,
# as 'sounddevice' tries to access the server's mic (which doesn't exist).
# For now, this code works if running locally.